**Tantangan Proyek: Analisis Data Penjualan Musik 🎵**


**Skenario:**

**Anda adalah seorang analis data di sebuah toko musik online. Anda diberi beberapa set data:**

1. tracks.csv: Berisi informasi tentang semua lagu yang tersedia (kolom: track_id, name, album_id, genre_id).

2. genres.csv: Daftar semua genre musik (kolom: genre_id, genre_name).

3. invoices_2022.csv: Data penjualan dari tahun 2022 (kolom: invoice_id, track_id, quantity).

4. invoices_2023.csv: Data penjualan dari tahun 2023 (kolom: invoice_id, track_id, quantity).

5. unpopular_genres.csv: Daftar genre yang penjualannya sedang menurun (kolom: genre_id, genre_name).

---

**Load Dataset**

In [19]:
import pandas as pd

In [20]:
df_tracks = pd.read_csv('tracks.csv')
df_tracks

,track_id,name,album_id,genre_id
0,101,Bohemian Rhapsody,1,1
1,102,Stairway to Heaven,2,1
2,103,Take Five,3,2
3,104,Enter Sandman,4,3
4,105,Billie Jean,5,4
5,106,Für Elise,6,5
6,107,Another One Bites the Dust,1,1
7,108,Unforgiven,4,3
8,109,Moonlight Sonata,6,5
9,110,Smooth Criminal,5,4


In [21]:
df_genres = pd.read_csv('genres.csv')
df_genres

,genre_id,genre_name
0,1,Rock
1,2,Jazz
2,3,Metal
3,4,Pop
4,5,Classical


In [22]:
df_invoice_2022 = pd.read_csv('invoices_2022.csv')
df_invoice_2022

,invoice_id,track_id,quantity
0,1,101,2
1,2,103,1
2,3,104,5
3,4,107,1
4,5,105,3


In [23]:
df_invoice_2023 = pd.read_csv('invoices_2023.csv')
df_invoice_2023

,invoice_id,track_id,quantity
0,6,101,1
1,7,102,2
2,8,104,1
3,9,110,4
4,10,108,2


In [24]:
df_unpopular_genres = pd.read_csv('unpopular_genres.csv')
df_unpopular_genres

,genre_id,genre_name
0,2,Jazz
1,5,Classical


----

**1. Gabungkan Data Penjualan:**

- Satukan invoices_2022.csv dan invoices_2023.csv menjadi satu DataFrame tunggal bernama all_invoices.
- Pastikan indeksnya berurutan dan tidak ada duplikasi.

In [28]:
#concat -> menggabungkan datset dengan metode ignore_index=True agar berurutan dan tidak ada duplikasi
all_invoice  = pd.concat([df_invoice_2022, df_invoice_2023], ignore_index=True)
all_invoice

,invoice_id,track_id,quantity
0,1,101,2
1,2,103,1
2,3,104,5
3,4,107,1
4,5,105,3
5,6,101,1
6,7,102,2
7,8,104,1
8,9,110,4
9,10,108,2


---

**2. Temukan Lagu yang Tidak Pernah Terjual:**

- Gunakan all_invoices untuk menemukan lagu-lagu dalam tracks.csv yang tidak pernah terjual sama sekali.
- Simpan hasilnya dalam DataFrame unsold_tracks. (Petunjuk: Operasi apa yang mengembalikan baris yang tidak cocok?)

In [30]:
# Anti Join -> menggabungkan dataset dengan mengembalikan baris yang tidak cocok atau tidak ada pasangannya
sold_track_ids = all_invoice['track_id'].unique()
unsold_tracks = df_tracks[~df_tracks['track_id'].isin(sold_track_ids)]
unsold_tracks 



,track_id,name,album_id,genre_id
5,106,Für Elise,6,5
8,109,Moonlight Sonata,6,5


---

**3. Analisis Genre Populer: (terjual)**

- Pertama, filter DataFrame tracks untuk hanya menyertakan lagu yang pernah terjual. Gunakan DataFrame all_invoices sebagai filter.
- Simpan hasilnya sebagai sold_tracks. (Petunjuk: Operasi apa yang menyaring baris berdasarkan kecocokan tanpa menambah kolom?)
- Gabungkan sold_tracks dengan genres.csv untuk mendapatkan nama genre untuk setiap lagu yang terjual.

In [31]:
# semi join -> menggabungkan dataset dengan mengembalikan baris yang  cocok tanpa menambahkan kolom apapun(apa adanya)

sold_track_ids_2 = all_invoice['track_id'].unique()
sold_tracks = df_tracks[df_tracks['track_id'].isin(sold_track_ids_2)]

sold_track_with_genres = sold_tracks.merge(df_genres, on='genre_id')

sold_track_with_genres

,track_id,name,album_id,genre_id,genre_name
0,101,Bohemian Rhapsody,1,1,Rock
1,102,Stairway to Heaven,2,1,Rock
2,103,Take Five,3,2,Jazz
3,104,Enter Sandman,4,3,Metal
4,105,Billie Jean,5,4,Pop
5,107,Another One Bites the Dust,1,1,Rock
6,108,Unforgiven,4,3,Metal
7,110,Smooth Criminal,5,4,Pop


---

**4. Verifikasi Integritas Data:**

- Saat menggabungkan sold_tracks dengan genres.csv, gunakan parameter validate untuk memastikan hubungan datanya benar (setiap lagu hanya memiliki satu genre). Asumsikan track_id unik di sold_tracks dan genre_id unik di genres.csv. Hubungan apa yang seharusnya Anda validasi?

In [38]:

sold_tracks_with_genres = sold_tracks.merge(
    df_genres,
    on='genre_id',
    validate='many_to_one' # Validasi hubungan many-to-one
)

sold_track_with_genres

,track_id,name,album_id,genre_id,genre_name
0,101,Bohemian Rhapsody,1,1,Rock
1,102,Stairway to Heaven,2,1,Rock
2,103,Take Five,3,2,Jazz
3,104,Enter Sandman,4,3,Metal
4,105,Billie Jean,5,4,Pop
5,107,Another One Bites the Dust,1,1,Rock
6,108,Unforgiven,4,3,Metal
7,110,Smooth Criminal,5,4,Pop


---


**5. Bonus - Genre yang Perlu Perhatian:**

- Gunakan hasil dari langkah 3 dan DataFrame unpopular_genres.csv untuk membuat daftar lagu-lagu terjual yang termasuk dalam kategori genre tidak populer.

In [41]:
# mencari lagu yang terjual tetapi masuk dalam kategori tidak populer dengan intersection/irisan
unpopuler_sold_track = sold_track_with_genres.merge(df_unpopular_genres, on='genre_name', suffixes=('_original', '_unpopular'))
unpopuler_sold_track

,track_id,name,album_id,genre_id_original,genre_name,genre_id_unpopular
0,103,Take Five,3,2,Jazz,2


----